In [1]:
from pymilvus import MilvusClient

host = "localhost"
port = "19530"

milvus_client = MilvusClient(
    host=host,
    port=port
)

In [2]:
from pymilvus import FieldSchema, DataType, CollectionSchema

VECTOR_LENGTH = 768 

id_field = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, description="Primary id")
text = FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=4096, description="Page text")
embedding_text = FieldSchema("embedding", dtype=DataType.FLOAT_VECTOR, dim=VECTOR_LENGTH, description="Embedded text")

fields = [id_field, text, embedding_text]

schema = CollectionSchema(fields=fields, auto_id=True, enable_dynamic_field=True, description="RAG Texts collection")

In [9]:
COLLECTION_NAME = "rag_texts_and_embeddings"

milvus_client.create_collection(
    collection_name=COLLECTION_NAME,
    schema=schema
)

index_params = milvus_client.prepare_index_params()

index_params.add_index(
    field_name="embedding", 
    index_type="HNSW",
    metric_type="L2",
    params={"M": 4, "efConstruction": 64}  # lower values for speed
) 

milvus_client.create_index(
    collection_name=COLLECTION_NAME,
    index_params=index_params
)

# checkout our collection
print(milvus_client.list_collections())

# describe our collection
print(milvus_client.describe_collection(COLLECTION_NAME))


['rag_texts_and_embeddings']
{'collection_name': 'rag_texts_and_embeddings', 'auto_id': True, 'num_shards': 1, 'description': 'RAG Texts collection', 'fields': [{'field_id': 100, 'name': 'id', 'description': 'Primary id', 'type': <DataType.INT64: 5>, 'params': {}, 'auto_id': True, 'is_primary': True}, {'field_id': 101, 'name': 'text', 'description': 'Page text', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 4096}}, {'field_id': 102, 'name': 'embedding', 'description': 'Embedded text', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}], 'functions': [], 'aliases': [], 'collection_id': 457750080042434562, 'consistency_level': 2, 'properties': {}, 'num_partitions': 1, 'enable_dynamic_field': True, 'created_timestamp': 457752224516800515}


In [4]:
#Its just for me. I use it in the case if I restart my project and need it for run the previous command without issues.
#It detects if the project 

COLLECTION_NAME = "rag_texts_and_embeddings"

if milvus_client.has_collection(COLLECTION_NAME):
    print(f"Dropping existing collection: {COLLECTION_NAME}")
    milvus_client.drop_collection(COLLECTION_NAME)
    print("Collection dropped successfully")
else:
    print("Collection doesn't exist")

Dropping existing collection: rag_texts_and_embeddings
Collection dropped successfully


In [10]:
# define data source and destination
## the document origin destination from which document will be downloaded 
pdf_url = "https://www.iab.org.pl/wp-content/uploads/2024/04/Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska.pdf"

## local destination of the document
file_name = "Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska.pdf"

## local destination of the processed document 
file_json = "Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska.json"

## local destination of the embedded pages of the document
embeddings_json = "Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska-Embeddings.json"

## local destination of all above local required files
data_dir = "./milvus_db/data"

In [6]:
import os
import requests

def download_pdf_data(pdf_url: str, file_name: str) -> None:
    response = requests.get(pdf_url, stream=True)
    with open(os.path.join(data_dir, file_name), "wb") as file:
        for block in response.iter_content(chunk_size=1024):
            if block:
                file.write(block)

download_pdf_data(pdf_url, file_name)

In [7]:
import fitz
import json


def extract_pdf_text(file_name, file_json):
    document = fitz.open(os.path.join(data_dir, file_name))
    pages = []

    for page_num in range(len(document)):
        page = document.load_page(page_num)
        page_text = page.get_text()
        pages.append({"page_num": page_num, "text": page_text})

    with open(os.path.join(data_dir, file_json), "w") as file:
        json.dump(pages, file, indent=4, ensure_ascii=False)

extract_pdf_text(file_name, file_json)

In [8]:
import torch
import numpy as np
from sentence_transformers import SentenceTransformer


def generate_embeddings(file_json, embeddings_json, model):
    pages = []
    with open(os.path.join(data_dir, file_json), "r") as file:
        data = json.load(file)

    for page in data:
        pages.append(page["text"])

    embeddings = model.encode(pages)

    embeddings_paginated = []
    for page_num in range(len(embeddings)):
        embeddings_paginated.append({"page_num": page_num, "embedding": embeddings[page_num].tolist()})

    with open(os.path.join(data_dir, embeddings_json), "w") as file:
        json.dump(embeddings_paginated, file, indent=4, ensure_ascii=False)

model_name = "ipipan/silver-retriever-base-v1.1"

#device = "cuda" if torch.cuda.is_available() else "cpu"
# Error from my console: (I dont know why it does not work with WSL now. I have to make a rsearch. It did work on windows earlier with the same GPU though)
# NVIDIA GeForce RTX 5070 Ti with CUDA capability sm_120 is not compatible with the current PyTorch installation.
# The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_70 sm_75 sm_80 sm_86 sm_90.
device = "cpu"
model = SentenceTransformer(model_name, device=device)
generate_embeddings(file_json, embeddings_json, model)

In [11]:
def insert_embeddings(file_json, embeddings_json, client=milvus_client):
    rows = []
    with open(os.path.join(data_dir, file_json), "r") as t_f, open(os.path.join(data_dir, embeddings_json), "r") as e_f:
        text_data, embedding_data = json.load(t_f), json.load(e_f)
        text_data =  list(map(lambda d: d["text"], text_data))
        embedding_data = list(map(lambda d: d["embedding"], embedding_data))
        
        for page, (text, embedding) in enumerate(zip(text_data, embedding_data)):
            rows.append({"text":text, "embedding": embedding})

    client.insert(collection_name="rag_texts_and_embeddings", data=rows)


insert_embeddings(file_json, embeddings_json)

# load inserted data into memory
milvus_client.load_collection("rag_texts_and_embeddings")

In [12]:
def search(model, query, client=milvus_client):
    embedded_query = model.encode(query).tolist()
    result = client.search(
        collection_name="rag_texts_and_embeddings", 
        data=[embedded_query], 
        limit=1,
        search_params={"metric_type": "L2"},
        output_fields=["text"]
    )
    return result

#result = search(model, query="Czym jest sztuczna inteligencja")

In [19]:
import os
import google.generativeai as genai

GOOGLE_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_KEY)
MODEL = "gemini-2.0-flash"
gemini_model = genai.GenerativeModel(MODEL)

def generate_response(prompt: str):
    try:
        # Send request to Gemini 2.0 Flash API and get the response
        response = gemini_model.generate_content(prompt)
        return response.text 
    except Exception as e:
        print(f"Error generating response: {e}")
        return None

In [14]:

def build_prompt(context: str, question: str) -> str:
    return f"""Na podstawie poniższych fragmentów dokumentu odpowiedz na pytanie.
    
    Kontekst:
    {context}
    
    Pytanie: {question}
    Odpowiedź:"""

def rag(query: str):
    # Retrieve context
    results = search(model, query=query)
    context = "\n".join(hit["entity"]["text"] for hit in results[0])
    
    # Generate response
    response = generate_response(build_prompt(context, query))
    return response



In [16]:
rag("Czym jest sztuczna inteligencja?")

'Sztuczna inteligencja (AI) to obszar informatyki, który skupia się na tworzeniu programów komputerowych zdolnych do wykonywania zadań, które wymagają ludzkiej inteligencji, takich jak rozpoznawanie wzorców, rozumienie języka naturalnego, podejmowanie decyzji, uczenie się i planowanie. Głównym celem AI jest stworzenie systemów, które są zdolne do myślenia i podejmowania decyzji na sposób przypominający ludzki. W potocznym rozumieniu, sztuczna inteligencja to coś (program, maszyna) symulujące inteligencję naturalną, ludzką.\n'

In [17]:
rag("Co to jest Gemini?")

'W tekście nie ma informacji na temat Gemini. Tekst opisuje Sztuczną Inteligencję Ogólnego Przeznaczenia (AGI).\n'

In [18]:
rag("Opowiedz mi o poczatkach sztucznej inteligencji")

'Historia sztucznej inteligencji sięga lat 50. XX wieku, kiedy to powstały pierwsze koncepcje i modele tego, co mogłoby się nią stać. Jednym z pionierów był Alan Turing, który sformułował test Turinga, mający na celu ocenę zdolności maszyny do inteligentnego zachowania na poziomie ludzkim.\n'